In [12]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\MARK3\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [13]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [14]:
img_height, img_width = (224,224)
batch_size = 32

train_data_dir = r"C:\\Users\\MARK3\\Downloads\\Omdena Ethiopia Coffee Diseases\\processed_data\\train"
valid_data_dir = r"C:\\Users\\MARK3\\Downloads\\Omdena Ethiopia Coffee Diseases\\processed_data\\val"
test_data_dir = r"C:\\Users\\MARK3\\Downloads\\Omdena Ethiopia Coffee Diseases\\processed_data\\test"

In [15]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range = 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip=True,
                                   validation_split= 0.4)
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size= batch_size,
class_mode = 'categorical',
subset='training') #set as training data


valid_generator = train_datagen.flow_from_directory(
valid_data_dir, #same directory as training data 
target_size = (img_height, img_width),
batch_size= batch_size,
class_mode = 'categorical',
subset='training')


Found 15955 images belonging to 3 classes.
Found 5318 images belonging to 3 classes.


In [16]:
test_generator = train_datagen.flow_from_directory(
test_data_dir,  #same as  training data
target_size = (img_height, img_width),
batch_size= batch_size,
class_mode = 'categorical',
subset='training')

Found 5322 images belonging to 3 classes.


In [ ]:
x,y =test_generator.next()
x.shape

In [ ]:
base_model = ResNet50(include_top=False , weights = 'imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs= base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    

model.compile(optimizer='adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(train_generator, epochs = 10)